In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import demoji
import spacy

ModuleNotFoundError: No module named 'demoji'

# Load DataSet and Data Dictionary

In [ ]:
# main_dir = 
Dataset_name = 'TechLabsDataset.csv' 
DataDictionary_name = 'TechLabsDataset_Dictionary.csv'

Dataset = pd.read_csv(os.path.join('./Data/',Dataset_name), index_col = 0)
DataDictionary = pd.read_csv(os.path.join('./Data/',DataDictionary_name), index_col = 0)
Dataset.head()

In [ ]:
review_df = pd.DataFrame(Dataset[~Dataset['review_EN'].isnull()]['review_EN'])
review_df.head()


In [ ]:
# find emo
def find_emo(text):
    # find emoji and add to list
    all_emoji = demoji.findall(text)
    emo = ''
    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            # list emo
            emo = emo+','+all_emoji[j]
    return emo


# replace emo with empty text
def replace_emo(text):
    # find emo and replce with empty text
    all_emoji = demoji.findall(text)

    if len(all_emoji)>0:
        for i,j in enumerate(all_emoji):
            text = text.replace(j,'')
    return text

In [ ]:
# add emo
review_df['emo'] = review_df['review_EN'].apply(lambda x: find_emo(x))

In [ ]:
# add review with no emo
review_df['review_no_emo'] = review_df['review_EN'].apply(lambda x: replace_emo(x))
# review_df[review_df['emo']!=''][['review_no_emo','emo']]

# NLP pre-processing on review_df['review_no_emo']


In [ ]:
# get clean lemmas
def get_clean_lemma(text):  
    text = [token.lemma_ for token in nlp(text.lower()) if
                  not token.is_punct
                  and not token.is_currency
                  and not token.is_digit
                  and not token.is_punct
    #               and not token.is_oov# is out of vocabulary
                  and not token.is_space
                  and not token.is_stop
                  and not token.like_num
                  and not token.pos_== 'PUNCT'
                          ]
    return text


# # get tags per token
# def get_tag_lemma(text): 
#     tag_list = [token.pos_ for token in nlp(text.lower()) if
#                   not token.is_punct
#                   and not token.is_currency
#                   and not token.is_digit
#                   and not token.is_punct
#     #               and not token.is_oov# is out of vocabulary
#                   and not token.is_space
#                   and not token.is_stop
#                   and not token.like_num
#                           ]
#     # ['NN','NNS','NNP','NNPS'] nouns
#     # ['JJ','JJR','JJS'] adjectives
#     # ['RB','RBR','RBS'] adverbs
#     # ['VB','VBD','VBG','VBN','VBP','VBZ'] verbs
#     return tag_list


In [ ]:

# check attribute of token in spacy: https://spacy.io/api/token

# list of stop words in spacy
# spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
# print('Number of stop words: %d' % len(spacy_stopwords))
# print('First ten stop words: %s' % list(spacy_stopwords))

nlp = spacy.load("en_core_web_sm")

review_df['text_lemmas'] = review_df['review_no_emo'].apply(lambda x: get_clean_lemma(x))
# review_df['lemmas_tags'] = review_df['review_no_emo'].apply(lambda x: get_tag_lemma(x))



In [ ]:

concatenate_all_tokens = sum(review_df['text_lemmas'].tolist(),[])
from collections import Counter


word_counts = Counter(concatenate_all_tokens)


In [ ]:
word_counts_df = pd.DataFrame.from_dict(word_counts, orient='index').reset_index()
word_counts_df.columns = ['word','#']
word_counts_df.sort_values('#',ascending = False, inplace = True)

In [ ]:
word_counts_df['tag'] = word_counts_df['word'].apply(lambda x: nlp(x)[0].pos_)

In [ ]:
np.unique(word_counts_df['tag'])

In [ ]:

word_counts_df[word_counts_df['tag']=='X']

In [ ]:
word_counts_df['tag_short'] = word_counts_df['tag'].apply(lambda x: tag_short(x))

In [ ]:
word_counts_df[word_counts_df['tag_short']=='adverbs']['word'].tolist()
['ea','p.m.','not','a.m.','ear','har','eurostar','tattoo','semi','ex-','right.only',
 'there.fast','gtv','bezeike','alphen','off.a','cbk']

In [ ]:
word_counts_df[word_counts_df['tag_short']=='verbs']['word'].tolist()
# ['ea','p.m.','not','a.m.','ear','har','eurostar','tattoo','semi','ex-','right.only',
#  'there.fast','gtv','bezeike','alphen','off.a','cbk']
['netherlands','cmr','ref','den','fl','tci','lug','b.v','kamado','luxembourg','cm','bo','bah','bio','m','nico',
'logistics&move','ist','lorentaweg','sweden','co','amir',
 'rabote','fajn','om','cove',
 'leadl''patrick',
 'toplogistiek','toitoi','swerve','lkw','aceo',
 'melden','ce',
 'satay',
 'manfre',]

In [ ]:
word_counts_df[word_counts_df['tag_short']=='nouns']